# Downloading parcellation and probability maps

`siibra` can facilitate structured, automatized and reprodicible download of parcellation maps. In this example, we demonstrate how to download all maximum probability and probabilistic maps of *Julich-Brain* - more then 300 structures available in different spaces. 

As often, we start by loading the human atlas.

In [1]:
import siibra


atlas = siibra.atlases.MULTILEVEL_HUMAN_ATLAS

[siibra:INFO] Version: 1.0a07
[siibra:WARNING] This is a development release. Use at your own risk.
[siibra:INFO] Please file bugs and issues at https://github.com/FZJ-INM1-BDA/siibra-python.
Loading preconfigured Atlas instances: 100%|██████████| 4/4 [00:00<00:00, 478.58Atlas/s]


Maps are delineations of brain regions in a particular coordinate space. We access regions from a parcellation object, and request maps by specifying the desired target space. In the simple case, we just fetch the first of possibly several labelled maps offered by the parcellation:

In [2]:
# request the parcellation object
julichbrain = atlas.get_parcellation("julich")

# request the map object for the desired space and type
lmap = julichbrain.get_map(space="mni152", maptype="labelled")

# fetch the (first available) actual image volume of this map.
img = lmap.fetch()
print(type(img))

Loading preconfigured Parcellation instances: 100%|██████████| 27/27 [00:00<00:00, 58.80Parcellation/s]
[siibra:INFO] Multiple elements matched the specification 'julich' - the first in order was chosen: Julich-Brain Cytoarchitectonic Atlas (v3.0.3)
Loading preconfigured Map instances: 100%|██████████| 58/58 [00:02<00:00, 26.20Map/s]
[siibra:WARNING] Map registry contains multiple classes: SparseMap, Map
Loading preconfigured Space instances: 100%|██████████| 9/9 [00:00<00:00, 117.59Space/s]


<class 'nibabel.nifti1.Nifti1Image'>


To fetch all available maps from the given parcellation, we iterate in two loops: 

- First loop: all available map types 
- Second loop: All maps that the parcellation defines for a given type

In [9]:
targetspace = atlas.get_space("mni152")

for maptype in siibra.MapType:

    print(f"Map type: {maptype.name}")

    parcellation_map = julichbrain.get_map(targetspace, maptype)
    fname_head = f"{julichbrain.key}-{targetspace.key}-{maptype.name}"

    for i, region in enumerate(parcellation_map.regions):

        # Check if this mapindex represents a region or not.
        # For Julich-Brain, this will be the case for the continuous map type,
        # which represents the probability maps.
        try:
            mapindex = parcellation_map.get_index(region)
            img = parcellation_map.fetch(index=mapindex)
            fname_tail = f"{region.key}"
        except Exception:
            fname_tail = f"volume_{mapindex.volume}-label_{mapindex.volume}{f'-fragment_{mapindex.fragment}' if mapindex.fragment else ''}"

        fname = siibra.commons.snake2camel(f"{fname_head}-{fname_tail}.nii.gz")
        print(f" - {fname}")
        img.to_filename(fname)

        # we stop after 5 maps in this example
        if i > 4:
            break

Map type: LABELLED
 - JulichBrainCytoarchitectonicAtlasV303-mni152Icbm2009cNonlinearAsymmetric-labelled-volume0-label0.nii.gz
 - JulichBrainCytoarchitectonicAtlasV303-mni152Icbm2009cNonlinearAsymmetric-labelled-volume0-label0.nii.gz
 - JulichBrainCytoarchitectonicAtlasV303-mni152Icbm2009cNonlinearAsymmetric-labelled-volume0-label0.nii.gz
 - JulichBrainCytoarchitectonicAtlasV303-mni152Icbm2009cNonlinearAsymmetric-labelled-volume0-label0.nii.gz
 - JulichBrainCytoarchitectonicAtlasV303-mni152Icbm2009cNonlinearAsymmetric-labelled-volume0-label0.nii.gz


[siibra:WARNING] Multiple maps are available in this specification of space, parcellation, and map type.
Choosing the first map from ['mni152 jba30 157regions statistical', 'mni152 jba30 175regions statistical'].


 - JulichBrainCytoarchitectonicAtlasV303-mni152Icbm2009cNonlinearAsymmetric-labelled-volume0-label0.nii.gz
Map type: STATISTICAL


Loading sparse index: 100%|██████████| 1849162/1849162 [00:06<00:00, 298223.27voxels/s]


 - JulichBrainCytoarchitectonicAtlasV303-mni152Icbm2009cNonlinearAsymmetric-statistical-volume0-label0.nii.gz
 - JulichBrainCytoarchitectonicAtlasV303-mni152Icbm2009cNonlinearAsymmetric-statistical-volume1-label1.nii.gz
 - JulichBrainCytoarchitectonicAtlasV303-mni152Icbm2009cNonlinearAsymmetric-statistical-volume2-label2.nii.gz
 - JulichBrainCytoarchitectonicAtlasV303-mni152Icbm2009cNonlinearAsymmetric-statistical-volume3-label3.nii.gz
 - JulichBrainCytoarchitectonicAtlasV303-mni152Icbm2009cNonlinearAsymmetric-statistical-volume4-label4.nii.gz
 - JulichBrainCytoarchitectonicAtlasV303-mni152Icbm2009cNonlinearAsymmetric-statistical-volume5-label5.nii.gz
